# Advanced Model Selection and Hyperparameter Tuning

This notebook demonstrates sophisticated model selection techniques and hyperparameter optimization strategies available in the sklearn-mastery project, including automated model selection, advanced optimization algorithms, and multi-objective optimization.

## Table of Contents
1. [Setup and Imports](#setup)
2. [Automated Model Selection](#automated)
3. [Advanced Hyperparameter Optimization](#optimization)
4. [Multi-Objective Optimization](#multi-objective)
5. [Bayesian Optimization](#bayesian)
6. [Population-Based Training](#population)
7. [Cross-Validation Strategies](#cross-validation)
8. [Model Selection Pipelines](#pipelines)

## 1. Setup and Imports {#setup}

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score,
    StratifiedKFold, GroupKFold, TimeSeriesSplit, validation_curve, learning_curve
)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform, randint
import time
import warnings
warnings.filterwarnings('ignore')

# Advanced ML imports
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern
from sklearn.inspection import permutation_importance
from sklearn.tree import export_text
from sklearn.base import BaseEstimator, ClassifierMixin
import joblib

# Results saving imports
import os
from pathlib import Path
import datetime
import json

In [ ]:
# Project imports
import sys
sys.path.append('../src')

from data.generators import SyntheticDataGenerator
from pipelines.model_selection import AdvancedModelSelector
from evaluation.metrics import ModelEvaluator
from evaluation.visualization import ModelVisualizationSuite
from utils.helpers import performance_timer
from utils.decorators import memory_profiler

# Configure plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette('husl')

print("✅ All imports successful!")

### Results Management Setup

In [ ]:
# Results saving setup for model selection
def setup_results_directories():
    """Create results directory structure if it doesn't exist."""
    base_dir = Path('../results')
    directories = [
        base_dir / 'figures',
        base_dir / 'models',
        base_dir / 'optimized_models',  # Specific for hyperparameter optimized models
        base_dir / 'pipelines',
        base_dir / 'experiments',
        base_dir / 'reports'
    ]
    
    for directory in directories:
        directory.mkdir(parents=True, exist_ok=True)
        print(f"📁 Created/verified directory: {directory}")
    
    return base_dir

def get_timestamp():
    """Get current timestamp for file naming."""
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

def save_figure(fig, name, description="", category="general", dpi=300):
    """Save figure with proper naming and metadata."""
    timestamp = get_timestamp()
    filename = f"{timestamp}_model_selection_{category}_{name}.png"
    filepath = results_dir / 'figures' / filename
    
    # Save figure
    fig.savefig(filepath, dpi=dpi, bbox_inches='tight', facecolor='white')
    
    # Save metadata
    metadata = {
        'filename': filename,
        'description': description,
        'category': category,
        'timestamp': timestamp,
        'notebook': '06_model_selection_tuning',
        'dpi': dpi
    }
    
    metadata_file = filepath.with_suffix('.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"💾 Saved figure: {filepath}")
    return filepath

def save_optimized_model(model, name, description="", optimization_results=None):
    """Save optimized model with comprehensive metadata."""
    timestamp = get_timestamp()
    filename = f"{timestamp}_optimized_{name}.joblib"
    filepath = results_dir / 'optimized_models' / filename
    
    # Save model
    joblib.dump(model, filepath, compress=3)
    
    # Save metadata
    metadata = {
        'filename': filename,
        'model_name': name,
        'description': description,
        'timestamp': timestamp,
        'notebook': '06_model_selection_tuning',
        'model_type': type(model).__name__,
        'optimization_results': optimization_results or {},
        'file_size_mb': filepath.stat().st_size / (1024*1024) if filepath.exists() else 0
    }
    
    # Add model-specific metadata
    if hasattr(model, 'best_params_'):
        metadata['best_params'] = model.best_params_
        metadata['best_score'] = model.best_score_
    
    if hasattr(model, 'cv_results_'):
        metadata['n_iterations'] = len(model.cv_results_['params'])
    
    metadata_file = filepath.with_suffix('.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2, default=str)
    
    print(f"💾 Saved optimized model: {filepath}")
    return filepath

def save_optimization_experiment(experiment_name, results, description=""):
    """Save optimization experiment results with detailed configuration."""
    timestamp = get_timestamp()
    filename = f"{timestamp}_optimization_{experiment_name}.json"
    filepath = results_dir / 'experiments' / filename
    
    experiment_data = {
        'experiment_name': experiment_name,
        'description': description,
        'timestamp': timestamp,
        'notebook': '06_model_selection_tuning',
        'results': results
    }
    
    with open(filepath, 'w') as f:
        json.dump(experiment_data, f, indent=2, default=str)
    
    print(f"💾 Saved optimization experiment: {filepath}")
    return filepath

def save_model_selection_report(content, report_name, format='txt'):
    """Save comprehensive model selection report."""
    timestamp = get_timestamp()
    filename = f"{timestamp}_model_selection_report_{report_name}.{format}"
    filepath = results_dir / 'reports' / filename
    
    if format == 'txt':
        with open(filepath, 'w') as f:
            f.write(content)
    elif format == 'json':
        with open(filepath, 'w') as f:
            json.dump(content, f, indent=2, default=str)
    
    print(f"💾 Saved report: {filepath}")
    return filepath

def save_pipeline(pipeline, name, description="", performance_metrics=None):
    """Save model selection pipeline with metadata."""
    timestamp = get_timestamp()
    filename = f"{timestamp}_pipeline_{name}.joblib"
    filepath = results_dir / 'pipelines' / filename
    
    # Save pipeline
    joblib.dump(pipeline, filepath, compress=3)
    
    # Save metadata
    metadata = {
        'filename': filename,
        'pipeline_name': name,
        'description': description,
        'timestamp': timestamp,
        'notebook': '06_model_selection_tuning',
        'pipeline_type': type(pipeline).__name__,
        'performance_metrics': performance_metrics or {},
        'file_size_mb': filepath.stat().st_size / (1024*1024) if filepath.exists() else 0
    }
    
    metadata_file = filepath.with_suffix('.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2, default=str)
    
    print(f"💾 Saved pipeline: {filepath}")
    return filepath

# Initialize results directories
results_dir = setup_results_directories()
print(f"📊 Results will be saved to: {results_dir}")

## 2. Automated Model Selection {#automated}

Let's explore automated model selection techniques that can find the best algorithm for a given dataset.

In [ ]:
# Generate datasets for model selection
print("🎯 Generating Datasets for Model Selection...")

generator = SyntheticDataGenerator(random_state=42)

# Dataset 1: Complex classification
X_complex, y_complex = generator.classification_dataset(
    n_samples=2000,
    n_features=25,
    n_informative=20,
    n_redundant=3,
    n_classes=3,
    n_clusters_per_class=2,
    class_sep=0.8
)

print(f"Complex classification dataset: {X_complex.shape}")
print(f"Class distribution: {np.bincount(y_complex)}")

# Dataset 2: Imbalanced classification
X_imbal, y_imbal = generator.imbalanced_classification(
    n_samples=1500,
    n_features=20,
    imbalance_ratio=0.1
)

print(f"Imbalanced classification dataset: {X_imbal.shape}")
print(f"Imbalanced class distribution: {np.bincount(y_imbal)}")

# Dataset 3: High-dimensional
X_highdim, y_highdim = generator.classification_dataset(
    n_samples=1000,
    n_features=100,
    n_informative=15,
    n_redundant=10,
    n_classes=2,
    class_sep=0.6
)

print(f"High-dimensional dataset: {X_highdim.shape}")
print(f"High-dim class distribution: {np.bincount(y_highdim)}")

# Split datasets
datasets = {
    'Complex': train_test_split(X_complex, y_complex, test_size=0.3, random_state=42, stratify=y_complex),
    'Imbalanced': train_test_split(X_imbal, y_imbal, test_size=0.3, random_state=42, stratify=y_imbal),
    'High-Dimensional': train_test_split(X_highdim, y_highdim, test_size=0.3, random_state=42, stratify=y_highdim)
}

print("\n✨ Datasets prepared for model selection!")

# Save dataset characteristics
dataset_info = {
    'complex': {
        'shape': X_complex.shape,
        'class_distribution': np.bincount(y_complex).tolist(),
        'n_features': X_complex.shape[1],
        'n_classes': len(np.unique(y_complex))
    },
    'imbalanced': {
        'shape': X_imbal.shape,
        'class_distribution': np.bincount(y_imbal).tolist(),
        'imbalance_ratio': min(np.bincount(y_imbal)) / max(np.bincount(y_imbal))
    },
    'high_dimensional': {
        'shape': X_highdim.shape,
        'class_distribution': np.bincount(y_highdim).tolist(),
        'dimensionality_ratio': X_highdim.shape[1] / X_highdim.shape[0]
    }
}

save_optimization_experiment('dataset_generation', dataset_info, 
                           'Generated datasets for model selection experiments')

### Automated Model Selection Implementation

In [ ]:
# Custom automated model selection implementation
print("🤖 Implementing Automated Model Selection...")

class AdvancedModelSelector:
    """Advanced automated model selection with comprehensive evaluation."""
    
    def __init__(self, cv_folds=5, scoring='accuracy', n_jobs=-1, verbose=True):
        self.cv_folds = cv_folds
        self.scoring = scoring
        self.n_jobs = n_jobs
        self.verbose = verbose
        
        # Define candidate models
        self.candidate_models = {
            'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
            'Random Forest': RandomForestClassifier(random_state=42),
            'Gradient Boosting': GradientBoostingClassifier(random_state=42),
            'SVM': SVC(random_state=42),
            'Neural Network': MLPClassifier(random_state=42, max_iter=500),
            'Gaussian Process': GaussianProcessClassifier(random_state=42)
        }
    
    def compare_models(self, X, y, task_type='classification'):
        """Compare all candidate models using cross-validation."""
        comparison_results = {}
        
        for name, model in self.candidate_models.items():
            if self.verbose:
                print(f"  Evaluating {name}...")
            
            try:
                start_time = time.time()
                
                # Cross-validation
                cv_scores = cross_val_score(model, X, y, cv=self.cv_folds, 
                                          scoring=self.scoring, n_jobs=self.n_jobs)
                
                evaluation_time = time.time() - start_time
                
                comparison_results[name] = {
                    'mean_score': cv_scores.mean(),
                    'std_score': cv_scores.std(),
                    'scores': cv_scores.tolist(),
                    'evaluation_time': evaluation_time
                }
                
                if self.verbose:
                    print(f"    Score: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
                
            except Exception as e:
                if self.verbose:
                    print(f"    Failed: {str(e)}")
                comparison_results[name] = {'error': str(e)}
        
        # Find best model
        valid_results = {k: v for k, v in comparison_results.items() if 'error' not in v}
        if valid_results:
            best_model_name = max(valid_results, key=lambda x: valid_results[x]['mean_score'])
            best_model = self.candidate_models[best_model_name]
            best_model.fit(X, y)  # Fit on full data
            
            return best_model, comparison_results
        else:
            return None, comparison_results

# Test automated model selection
model_selector = AdvancedModelSelector(
    cv_folds=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=True
)

selection_results = {}

for dataset_name, (X_train, X_test, y_train, y_test) in datasets.items():
    print(f"\n--- Automated Selection for {dataset_name} Dataset ---")
    
    try:
        start_time = time.time()
        
        # Run automated model selection
        best_model, comparison_results = model_selector.compare_models(
            X_train, y_train, task_type='classification'
        )
        
        selection_time = time.time() - start_time
        
        if best_model:
            # Test best model
            test_score = best_model.score(X_test, y_test)
            y_pred = best_model.predict(X_test)
            
            selection_results[dataset_name] = {
                'best_model': best_model.__class__.__name__,
                'best_cv_score': max([result['mean_score'] for result in comparison_results.values() 
                                    if 'error' not in result]),
                'test_score': test_score,
                'selection_time': selection_time,
                'all_results': comparison_results,
                'y_pred': y_pred,
                'y_test': y_test
            }
            
            print(f"Best Model: {best_model.__class__.__name__}")
            print(f"Best CV Score: {selection_results[dataset_name]['best_cv_score']:.4f}")
            print(f"Test Score: {test_score:.4f}")
            print(f"Selection Time: {selection_time:.2f}s")
            
            # Save the best model for each dataset
            optimization_results = {
                'best_cv_score': selection_results[dataset_name]['best_cv_score'],
                'test_score': test_score,
                'selection_time': selection_time,
                'dataset': dataset_name,
                'all_model_scores': {name: result.get('mean_score', 0) 
                                   for name, result in comparison_results.items() 
                                   if 'error' not in result}
            }
            
            save_optimized_model(best_model, f"autoselected_{dataset_name.lower()}_best_{best_model.__class__.__name__.lower()}",
                               f"Best model selected automatically for {dataset_name} dataset",
                               optimization_results)
        else:
            print("❌ No valid models found")
            selection_results[dataset_name] = {'error': 'No valid models'}
        
    except Exception as e:
        print(f"❌ Failed: {str(e)}")
        selection_results[dataset_name] = {'error': str(e)}

print("\n✨ Automated model selection complete!")

# Save comprehensive model selection results
model_selection_summary = {dataset: {
    'best_model': results.get('best_model', 'Failed'),
    'best_cv_score': results.get('best_cv_score', 0),
    'test_score': results.get('test_score', 0),
    'selection_time': results.get('selection_time', 0)
} for dataset, results in selection_results.items() if 'error' not in results}

save_optimization_experiment('automated_model_selection', model_selection_summary,
                           'Results from automated model selection across different datasets')

### Model Selection Visualization

In [ ]:
# Visualize automated model selection results
print("📊 Visualizing Automated Model Selection Results...")

if selection_results:
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Best models by dataset
    dataset_names = []
    best_models = []
    test_scores = []
    
    for dataset, result in selection_results.items():
        if 'error' not in result:
            dataset_names.append(dataset)
            best_models.append(result['best_model'])
            test_scores.append(result['test_score'])
    
    if dataset_names:
        colors = plt.cm.Set3(np.linspace(0, 1, len(set(best_models))))
        model_colors = {model: colors[i] for i, model in enumerate(set(best_models))}
        bar_colors = [model_colors[model] for model in best_models]
        
        bars = axes[0, 0].bar(dataset_names, test_scores, color=bar_colors, alpha=0.7)
        axes[0, 0].set_title('Best Models by Dataset')
        axes[0, 0].set_ylabel('Test Accuracy')
        axes[0, 0].set_ylim(0, 1)
        
        # Add model names on bars
        for bar, model, score in zip(bars, best_models, test_scores):
            axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                           model, ha='center', va='bottom', rotation=45, fontsize=8)
            axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height()/2, 
                           f'{score:.3f}', ha='center', va='center', fontweight='bold')
        
        # 2. Selection time comparison
        selection_times = [selection_results[name]['selection_time'] for name in dataset_names
                          if 'selection_time' in selection_results[name]]
        
        if selection_times:
            axes[0, 1].bar(dataset_names, selection_times, color='lightcoral', alpha=0.7)
            axes[0, 1].set_title('Model Selection Time')
            axes[0, 1].set_ylabel('Time (seconds)')
            
            for i, (name, time_val) in enumerate(zip(dataset_names, selection_times)):
                axes[0, 1].text(i, time_val + max(selection_times) * 0.02, 
                               f'{time_val:.1f}s', ha='center', va='bottom')
        
        # 3. Model performance comparison for first dataset
        if dataset_names and 'all_results' in selection_results[dataset_names[0]]:
            first_dataset = dataset_names[0]
            all_results = selection_results[first_dataset]['all_results']
            
            model_names = [name for name, result in all_results.items() if 'error' not in result]
            cv_scores = [all_results[model]['mean_score'] for model in model_names]
            cv_stds = [all_results[model]['std_score'] for model in model_names]
            
            bars = axes[1, 0].bar(range(len(model_names)), cv_scores, 
                                 yerr=cv_stds, capsize=5, color='lightgreen', alpha=0.7)
            axes[1, 0].set_title(f'Model Comparison - {first_dataset} Dataset')
            axes[1, 0].set_ylabel('CV Accuracy')
            axes[1, 0].set_xticks(range(len(model_names)))
            axes[1, 0].set_xticklabels(model_names, rotation=45, ha='right')
            
            # Highlight best model
            best_idx = np.argmax(cv_scores)
            bars[best_idx].set_color('gold')
            bars[best_idx].set_alpha(0.9)
        
        # 4. Performance vs dataset characteristics
        dataset_characteristics = {
            'Complex': {'n_features': 25, 'n_classes': 3, 'imbalance': 1.0},
            'Imbalanced': {'n_features': 20, 'n_classes': 2, 'imbalance': 0.1},
            'High-Dimensional': {'n_features': 100, 'n_classes': 2, 'imbalance': 1.0}
        }
        
        if len(dataset_names) >= 2:
            x_vals = [dataset_characteristics[name]['n_features'] for name in dataset_names]
            y_vals = test_scores
            
            scatter = axes[1, 1].scatter(x_vals, y_vals, c=range(len(dataset_names)), 
                                       cmap='viridis', s=100, alpha=0.7)
            
            for i, (x, y, name) in enumerate(zip(x_vals, y_vals, dataset_names)):
                axes[1, 1].annotate(name, (x, y), xytext=(5, 5), 
                                  textcoords='offset points', fontsize=9)
            
            axes[1, 1].set_xlabel('Number of Features')
            axes[1, 1].set_ylabel('Test Accuracy')
            axes[1, 1].set_title('Performance vs Dataset Complexity')
            axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save automated selection visualization
    save_figure(fig, 'automated_model_selection_results',
               'Comprehensive results from automated model selection across datasets', 'selection')
    plt.show()
    
    # Summary table
    print("\n📊 Automated Model Selection Summary:")
    print("=" * 80)
    print(f"{'Dataset':<15} {'Best Model':<20} {'CV Score':<10} {'Test Score':<10} {'Time(s)':<10}")
    print("=" * 80)
    
    for dataset in dataset_names:
        result = selection_results[dataset]
        print(f"{dataset:<15} {result['best_model']:<20} {result['best_cv_score']:<10.4f} "
              f"{result['test_score']:<10.4f} {result.get('selection_time', 0):<10.2f}")
    
    print("=" * 80)

print("\n✨ Automated model selection visualization complete!")

## 3. Advanced Hyperparameter Optimization {#optimization}

Let's explore sophisticated hyperparameter optimization techniques beyond grid and random search.

In [ ]:
# Advanced hyperparameter optimization
print("⚙️ Advanced Hyperparameter Optimization...")

# Use the complex dataset for optimization
X_train, X_test, y_train, y_test = datasets['Complex']

# Define models and parameter spaces
optimization_tasks = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'param_space': {
            'n_estimators': randint(50, 200),
            'max_depth': randint(3, 20),
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2', 0.5, 0.8]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(random_state=42),
        'param_space': {
            'n_estimators': randint(50, 150),
            'learning_rate': uniform(0.01, 0.2),
            'max_depth': randint(3, 10),
            'subsample': uniform(0.6, 0.4),
            'min_samples_split': randint(2, 15)
        }
    },
    'SVM': {
        'model': SVC(random_state=42),
        'param_space': {
            'C': uniform(0.1, 10),
            'gamma': ['scale', 'auto'] + list(uniform(0.001, 1).rvs(3)),
            'kernel': ['rbf', 'poly', 'sigmoid']
        }
    }
}

# Compare different optimization strategies
optimization_strategies = {
    'Random Search': 'random',
    'Grid Search': 'grid'
}

optimization_results = {}

for model_name, task in optimization_tasks.items():
    print(f"\n--- Optimizing {model_name} ---")
    
    optimization_results[model_name] = {}
    
    for strategy_name, strategy_type in optimization_strategies.items():
        print(f"\n  {strategy_name}:")
        
        try:
            start_time = time.time()
            
            if strategy_type == 'random':
                # Random search
                search = RandomizedSearchCV(
                    estimator=task['model'],
                    param_distributions=task['param_space'],
                    n_iter=50,
                    cv=5,
                    scoring='accuracy',
                    n_jobs=-1,
                    random_state=42
                )
                search.fit(X_train, y_train)
                
            elif strategy_type == 'grid':
                # Grid search (with reduced parameter space)
                grid_params = {}
                for param, values in task['param_space'].items():
                    if hasattr(values, 'rvs'):  # Continuous distribution
                        if param in ['learning_rate', 'subsample']:
                            grid_params[param] = [0.01, 0.1, 0.2]
                        elif param == 'C':
                            grid_params[param] = [0.1, 1, 10]
                        else:
                            grid_params[param] = values.rvs(3, random_state=42)
                    elif isinstance(values, list):
                        grid_params[param] = values[:3]  # Limit to first 3
                    else:
                        grid_params[param] = [values.rvs(1, random_state=42)[0] for _ in range(3)]
                
                search = GridSearchCV(
                    estimator=task['model'],
                    param_grid=grid_params,
                    cv=5,
                    scoring='accuracy',
                    n_jobs=-1
                )
                search.fit(X_train, y_train)
            
            optimization_time = time.time() - start_time
            
            # Evaluate best model
            best_model = search.best_estimator_
            test_score = best_model.score(X_test, y_test)
            
            optimization_results[model_name][strategy_name] = {
                'best_params': search.best_params_,
                'best_cv_score': search.best_score_,
                'test_score': test_score,
                'optimization_time': optimization_time,
                'n_iterations': getattr(search, 'n_iter', len(search.cv_results_['params']))
            }
            
            print(f"    Best CV Score: {search.best_score_:.4f}")
            print(f"    Test Score: {test_score:.4f}")
            print(f"    Optimization Time: {optimization_time:.2f}s")
            print(f"    Best Params: {search.best_params_}")
            
            # Save optimized model
            opt_results = {
                'strategy': strategy_name,
                'best_cv_score': search.best_score_,
                'test_score': test_score,
                'optimization_time': optimization_time,
                'n_iterations': optimization_results[model_name][strategy_name]['n_iterations'],
                'search_space_size': len(task['param_space'])
            }
            
            save_optimized_model(search, f"{strategy_type}_{model_name.lower().replace(' ', '_')}",
                               f"{model_name} optimized using {strategy_name}", opt_results)
            
        except Exception as e:
            print(f"    ❌ Failed: {str(e)}")

print("\n✨ Advanced hyperparameter optimization complete!")

# Save optimization comparison results
optimization_summary = {}
for model_name, strategies in optimization_results.items():
    optimization_summary[model_name] = {}
    for strategy_name, results in strategies.items():
        optimization_summary[model_name][strategy_name] = {
            'best_cv_score': results['best_cv_score'],
            'test_score': results['test_score'],
            'optimization_time': results['optimization_time'],
            'n_iterations': results['n_iterations']
        }

save_optimization_experiment('hyperparameter_optimization_comparison', optimization_summary,
                           'Comparison of different hyperparameter optimization strategies')

### Optimization Results Visualization

In [ ]:
# Visualize optimization results
print("📊 Visualizing Optimization Results...")

if optimization_results:
    # Prepare data for visualization
    comparison_data = []
    
    for model_name, strategies in optimization_results.items():
        for strategy_name, result in strategies.items():
            comparison_data.append({
                'Model': model_name,
                'Strategy': strategy_name,
                'CV_Score': result['best_cv_score'],
                'Test_Score': result['test_score'],
                'Time': result['optimization_time'],
                'Iterations': result['n_iterations']
            })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Test score comparison
    sns.barplot(data=comparison_df, x='Model', y='Test_Score', hue='Strategy', ax=axes[0, 0])
    axes[0, 0].set_title('Test Score by Optimization Strategy')
    axes[0, 0].set_ylabel('Test Accuracy')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].legend(title='Strategy')
    
    # 2. Optimization time comparison
    sns.barplot(data=comparison_df, x='Model', y='Time', hue='Strategy', ax=axes[0, 1])
    axes[0, 1].set_title('Optimization Time by Strategy')
    axes[0, 1].set_ylabel('Time (seconds)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].legend(title='Strategy')
    
    # 3. Efficiency analysis (Performance vs Time)
    for strategy in comparison_df['Strategy'].unique():
        strategy_data = comparison_df[comparison_df['Strategy'] == strategy]
        axes[1, 0].scatter(strategy_data['Time'], strategy_data['Test_Score'], 
                          label=strategy, s=100, alpha=0.7)
    
    axes[1, 0].set_xlabel('Optimization Time (seconds)')
    axes[1, 0].set_ylabel('Test Accuracy')
    axes[1, 0].set_title('Optimization Efficiency (Performance vs Time)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Strategy ranking by model
    strategy_rankings = []
    for model in comparison_df['Model'].unique():
        model_data = comparison_df[comparison_df['Model'] == model]
        ranked = model_data.sort_values('Test_Score', ascending=False)
        for i, (_, row) in enumerate(ranked.iterrows()):
            strategy_rankings.append({
                'Model': model,
                'Strategy': row['Strategy'],
                'Rank': i + 1
            })
    
    ranking_df = pd.DataFrame(strategy_rankings)
    ranking_pivot = ranking_df.pivot(index='Strategy', columns='Model', values='Rank')
    
    sns.heatmap(ranking_pivot, annot=True, cmap='RdYlGn_r', ax=axes[1, 1], 
                cbar_kws={'label': 'Rank (1=Best)'})
    axes[1, 1].set_title('Strategy Rankings by Model')
    axes[1, 1].set_xlabel('Model')
    axes[1, 1].set_ylabel('Optimization Strategy')
    
    plt.tight_layout()
    
    # Save optimization comparison figure
    save_figure(fig, 'hyperparameter_optimization_comparison',
               'Comprehensive comparison of hyperparameter optimization strategies', 'optimization')
    plt.show()
    
    # Summary table
    print("\n📊 Hyperparameter Optimization Summary:")
    print("=" * 90)
    print(f"{'Model':<20} {'Strategy':<15} {'CV Score':<10} {'Test Score':<10} {'Time(s)':<10} {'Iters':<8}")
    print("=" * 90)
    
    for _, row in comparison_df.iterrows():
        print(f"{row['Model']:<20} {row['Strategy']:<15} {row['CV_Score']:<10.4f} "
              f"{row['Test_Score']:<10.4f} {row['Time']:<10.2f} {row['Iterations']:<8}")
    
    print("=" * 90)
    
    # Best strategy analysis
    print("\n🏆 Best Strategy Analysis:")
    for model in comparison_df['Model'].unique():
        model_data = comparison_df[comparison_df['Model'] == model]
        best_row = model_data.loc[model_data['Test_Score'].idxmax()]
        print(f"  {model}: {best_row['Strategy']} (Score: {best_row['Test_Score']:.4f}, Time: {best_row['Time']:.1f}s)")

print("\n✨ Optimization results visualization complete!")

## 4. Multi-Objective Optimization {#multi-objective}

Exploring optimization techniques that balance multiple objectives like accuracy, speed, and model complexity.

In [ ]:
# Multi-objective optimization
print("🎯 Multi-Objective Optimization...")

class MultiObjectiveOptimizer:
    """Multi-objective hyperparameter optimization balancing performance and complexity."""
    
    def __init__(self, weights={'accuracy': 0.6, 'speed': 0.2, 'simplicity': 0.2}):
        self.weights = weights
        self.results = []
    
    def evaluate_configuration(self, model, params, X_train, y_train, X_val, y_val):
        """Evaluate a model configuration on multiple objectives."""
        try:
            # Set parameters
            model.set_params(**params)
            
            # Measure training time
            start_time = time.time()
            model.fit(X_train, y_train)
            training_time = time.time() - start_time
            
            # Measure prediction time
            start_time = time.time()
            predictions = model.predict(X_val)
            prediction_time = time.time() - start_time
            
            # Calculate metrics
            accuracy = accuracy_score(y_val, predictions)
            
            # Speed score (inverse of total time, normalized)
            total_time = training_time + prediction_time
            speed_score = 1.0 / (1.0 + total_time)  # Bounded between 0 and 1
            
            # Simplicity score (based on model complexity)
            simplicity_score = self._calculate_simplicity(model, params)
            
            # Combined score
            combined_score = (
                self.weights['accuracy'] * accuracy +
                self.weights['speed'] * speed_score +
                self.weights['simplicity'] * simplicity_score
            )
            
            return {
                'params': params,
                'accuracy': accuracy,
                'speed_score': speed_score,
                'simplicity_score': simplicity_score,
                'combined_score': combined_score,
                'training_time': training_time,
                'prediction_time': prediction_time
            }
            
        except Exception as e:
            return None
    
    def _calculate_simplicity(self, model, params):
        """Calculate simplicity score based on model parameters."""
        if hasattr(model, 'n_estimators'):
            # For ensemble methods, fewer estimators = more simple
            n_est = params.get('n_estimators', getattr(model, 'n_estimators', 100))
            return max(0, 1.0 - (n_est - 50) / 150)  # Normalize to 0-1
        
        elif hasattr(model, 'C'):
            # For SVM, smaller C = more simple
            C = params.get('C', getattr(model, 'C', 1.0))
            return max(0, 1.0 - np.log10(C + 1) / 2)  # Normalize to 0-1
        
        elif hasattr(model, 'hidden_layer_sizes'):
            # For neural networks, fewer/smaller layers = more simple
            layers = params.get('hidden_layer_sizes', getattr(model, 'hidden_layer_sizes', (100,)))
            if isinstance(layers, tuple):
                total_neurons = sum(layers)
                return max(0, 1.0 - (total_neurons - 50) / 500)  # Normalize to 0-1
        
        return 0.5  # Default moderate simplicity
    
    def optimize(self, model_class, param_distributions, X_train, y_train, X_val, y_val, n_trials=30):
        """Perform multi-objective optimization."""
        print(f"  Running multi-objective optimization with {n_trials} trials...")
        
        self.results = []
        
        for trial in range(n_trials):
            # Sample parameters
            params = {}
            for param_name, distribution in param_distributions.items():
                if hasattr(distribution, 'rvs'):
                    params[param_name] = distribution.rvs()
                elif isinstance(distribution, list):
                    params[param_name] = np.random.choice(distribution)
                else:
                    params[param_name] = distribution
            
            # Evaluate configuration
            model = model_class(random_state=42)
            result = self.evaluate_configuration(model, params, X_train, y_train, X_val, y_val)
            
            if result:
                self.results.append(result)
        
        # Sort by combined score
        self.results.sort(key=lambda x: x['combined_score'], reverse=True)
        
        return self.results

# Test multi-objective optimization
X_train, X_test, y_train, y_test = datasets['Complex']
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=42, stratify=y_train
)

multi_obj_results = {}

# Define optimization tasks
multi_obj_tasks = {
    'Random Forest': {
        'model_class': RandomForestClassifier,
        'param_distributions': {
            'n_estimators': randint(10, 100),
            'max_depth': randint(3, 15),
            'min_samples_split': randint(2, 10)
        }
    },
    'Gradient Boosting': {
        'model_class': GradientBoostingClassifier,
        'param_distributions': {
            'n_estimators': randint(10, 80),
            'learning_rate': uniform(0.05, 0.25),
            'max_depth': randint(3, 8)
        }
    }
}

# Test different weight configurations
weight_configs = {
    'Balanced': {'accuracy': 0.5, 'speed': 0.25, 'simplicity': 0.25},
    'Accuracy-Focused': {'accuracy': 0.8, 'speed': 0.1, 'simplicity': 0.1},
    'Speed-Focused': {'accuracy': 0.4, 'speed': 0.5, 'simplicity': 0.1},
    'Simplicity-Focused': {'accuracy': 0.4, 'speed': 0.1, 'simplicity': 0.5}
}

for model_name, task in multi_obj_tasks.items():
    print(f"\n--- Multi-Objective Optimization: {model_name} ---")
    multi_obj_results[model_name] = {}
    
    for weight_name, weights in weight_configs.items():
        print(f"\n  Weight Configuration: {weight_name}")
        
        optimizer = MultiObjectiveOptimizer(weights=weights)
        results = optimizer.optimize(
            task['model_class'], 
            task['param_distributions'],
            X_train_sub, y_train_sub, X_val, y_val,
            n_trials=20
        )
        
        if results:
            best_result = results[0]
            print(f"    Best Combined Score: {best_result['combined_score']:.4f}")
            print(f"    Accuracy: {best_result['accuracy']:.4f}")
            print(f"    Speed Score: {best_result['speed_score']:.4f}")
            print(f"    Simplicity Score: {best_result['simplicity_score']:.4f}")
            print(f"    Best Params: {best_result['params']}")
            
            multi_obj_results[model_name][weight_name] = {
                'best_result': best_result,
                'all_results': results[:5]  # Top 5 results
            }

print("\n✨ Multi-objective optimization complete!")

# Save multi-objective results
multi_obj_summary = {}
for model_name, weight_configs in multi_obj_results.items():
    multi_obj_summary[model_name] = {}
    for weight_name, results in weight_configs.items():
        best = results['best_result']
        multi_obj_summary[model_name][weight_name] = {
            'combined_score': best['combined_score'],
            'accuracy': best['accuracy'],
            'speed_score': best['speed_score'],
            'simplicity_score': best['simplicity_score'],
            'training_time': best['training_time'],
            'best_params': best['params']
        }

save_optimization_experiment('multi_objective_optimization', multi_obj_summary,
                           'Results from multi-objective hyperparameter optimization')

### Multi-Objective Results Visualization

In [ ]:
# Visualize multi-objective optimization results
print("📊 Visualizing Multi-Objective Optimization Results...")

if multi_obj_results:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Prepare data for visualization
    plot_data = []
    for model_name, weight_configs in multi_obj_results.items():
        for weight_name, results in weight_configs.items():
            best = results['best_result']
            plot_data.append({
                'Model': model_name,
                'Weight_Config': weight_name,
                'Combined_Score': best['combined_score'],
                'Accuracy': best['accuracy'],
                'Speed_Score': best['speed_score'],
                'Simplicity_Score': best['simplicity_score']
            })
    
    plot_df = pd.DataFrame(plot_data)
    
    # 1. Combined scores by weight configuration
    sns.barplot(data=plot_df, x='Weight_Config', y='Combined_Score', hue='Model', ax=axes[0, 0])
    axes[0, 0].set_title('Combined Scores by Weight Configuration')
    axes[0, 0].set_ylabel('Combined Score')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].legend(title='Model')
    
    # 2. Accuracy vs Speed trade-off
    for model in plot_df['Model'].unique():
        model_data = plot_df[plot_df['Model'] == model]
        axes[0, 1].scatter(model_data['Speed_Score'], model_data['Accuracy'], 
                          label=model, s=100, alpha=0.7)
    
    axes[0, 1].set_xlabel('Speed Score')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Accuracy vs Speed Trade-off')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Radar chart for first model
    if multi_obj_results:
        first_model = list(multi_obj_results.keys())[0]
        weight_configs = list(multi_obj_results[first_model].keys())
        
        # Prepare radar chart data
        categories = ['Accuracy', 'Speed', 'Simplicity']
        angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle
        
        ax = plt.subplot(2, 2, 3, projection='polar')
        
        for i, config_name in enumerate(weight_configs[:3]):  # Limit to 3 configs for clarity
            config_data = multi_obj_results[first_model][config_name]['best_result']
            values = [
                config_data['accuracy'],
                config_data['speed_score'],
                config_data['simplicity_score']
            ]
            values += values[:1]  # Complete the circle
            
            ax.plot(angles, values, 'o-', linewidth=2, label=config_name)
            ax.fill(angles, values, alpha=0.25)
        
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories)
        ax.set_ylim(0, 1)
        ax.set_title(f'Multi-Objective Performance - {first_model}')
        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    # 4. Pareto frontier analysis
    if len(plot_df) > 0:
        # Use accuracy and speed for Pareto analysis
        accuracy_vals = plot_df['Accuracy'].values
        speed_vals = plot_df['Speed_Score'].values
        
        # Find Pareto frontier
        pareto_points = []
        for i in range(len(accuracy_vals)):
            is_pareto = True
            for j in range(len(accuracy_vals)):
                if i != j:
                    if (accuracy_vals[j] >= accuracy_vals[i] and speed_vals[j] >= speed_vals[i] and
                        (accuracy_vals[j] > accuracy_vals[i] or speed_vals[j] > speed_vals[i])):
                        is_pareto = False
                        break
            if is_pareto:
                pareto_points.append(i)
        
        # Plot all points
        axes[1, 1].scatter(speed_vals, accuracy_vals, alpha=0.6, s=50, color='lightblue')
        
        # Highlight Pareto frontier
        if pareto_points:
            pareto_acc = [accuracy_vals[i] for i in pareto_points]
            pareto_speed = [speed_vals[i] for i in pareto_points]
            axes[1, 1].scatter(pareto_speed, pareto_acc, color='red', s=100, 
                             label='Pareto Frontier', alpha=0.8)
        
        axes[1, 1].set_xlabel('Speed Score')
        axes[1, 1].set_ylabel('Accuracy')
        axes[1, 1].set_title('Pareto Frontier Analysis')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save multi-objective visualization
    save_figure(fig, 'multi_objective_optimization_results',
               'Multi-objective optimization results showing trade-offs between objectives', 'multi_objective')
    plt.show()
    
    # Summary table
    print("\n📊 Multi-Objective Optimization Summary:")
    print("=" * 100)
    print(f"{'Model':<15} {'Config':<18} {'Combined':<10} {'Accuracy':<10} {'Speed':<10} {'Simplicity':<10}")
    print("=" * 100)
    
    for _, row in plot_df.iterrows():
        print(f"{row['Model']:<15} {row['Weight_Config']:<18} {row['Combined_Score']:<10.4f} "
              f"{row['Accuracy']:<10.4f} {row['Speed_Score']:<10.4f} {row['Simplicity_Score']:<10.4f}")
    
    print("=" * 100)

print("\n✨ Multi-objective optimization visualization complete!")

## 5. Bayesian Optimization {#bayesian}

Implementing Bayesian optimization for efficient hyperparameter search.

In [ ]:
# Bayesian optimization implementation
print("🔬 Bayesian Optimization...")

class SimpleBayesianOptimizer:
    """Simplified Bayesian optimization using Gaussian Process."""
    
    def __init__(self, n_initial_points=5, n_calls=20):
        self.n_initial_points = n_initial_points
        self.n_calls = n_calls
        self.X_observed = []
        self.y_observed = []
        self.gp = None
    
    def _sample_parameters(self, param_space):
        """Sample parameters from the parameter space."""
        params = {}
        for param_name, param_range in param_space.items():
            if isinstance(param_range, tuple) and len(param_range) == 2:
                # Continuous parameter
                params[param_name] = np.random.uniform(param_range[0], param_range[1])
            elif isinstance(param_range, list):
                # Categorical parameter
                params[param_name] = np.random.choice(param_range)
            else:
                params[param_name] = param_range
        return params
    
    def _params_to_vector(self, params, param_space):
        """Convert parameter dictionary to vector for GP."""
        vector = []
        for param_name in sorted(param_space.keys()):
            if param_name in params:
                value = params[param_name]
                param_range = param_space[param_name]
                
                if isinstance(param_range, tuple):
                    # Normalize continuous parameters to [0, 1]
                    normalized = (value - param_range[0]) / (param_range[1] - param_range[0])
                    vector.append(normalized)
                elif isinstance(param_range, list):
                    # One-hot encode categorical parameters
                    one_hot = [1 if v == value else 0 for v in param_range]
                    vector.extend(one_hot)
        return np.array(vector)
    
    def _vector_to_params(self, vector, param_space):
        """Convert vector back to parameter dictionary."""
        params = {}
        idx = 0
        
        for param_name in sorted(param_space.keys()):
            param_range = param_space[param_name]
            
            if isinstance(param_range, tuple):
                # Denormalize continuous parameters
                normalized_value = vector[idx]
                value = param_range[0] + normalized_value * (param_range[1] - param_range[0])
                
                # Handle integer parameters
                if param_name in ['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf']:
                    value = int(round(value))
                
                params[param_name] = value
                idx += 1
            elif isinstance(param_range, list):
                # Decode categorical parameters
                one_hot_length = len(param_range)
                one_hot = vector[idx:idx + one_hot_length]
                selected_idx = np.argmax(one_hot)
                params[param_name] = param_range[selected_idx]
                idx += one_hot_length
        
        return params
    
    def _acquisition_function(self, X_candidate, gp):
        """Expected Improvement acquisition function."""
        try:
            mu, sigma = gp.predict(X_candidate.reshape(1, -1), return_std=True)
            
            if len(self.y_observed) == 0:
                return 0
            
            # Expected Improvement
            f_best = max(self.y_observed)
            xi = 0.01  # Exploration parameter
            
            if sigma[0] > 0:
                z = (mu[0] - f_best - xi) / sigma[0]
                ei = (mu[0] - f_best - xi) * norm.cdf(z) + sigma[0] * norm.pdf(z)
                return ei
            else:
                return 0
                
        except Exception:
            return 0
    
    def optimize(self, objective_function, param_space):
        """Perform Bayesian optimization."""
        from scipy.stats import norm
        from sklearn.gaussian_process import GaussianProcessRegressor
        from sklearn.gaussian_process.kernels import Matern
        
        print(f"  Starting Bayesian optimization with {self.n_calls} iterations...")
        
        # Initial random exploration
        for i in range(self.n_initial_points):
            params = self._sample_parameters(param_space)
            score = objective_function(params)
            
            if score is not None:
                X_vec = self._params_to_vector(params, param_space)
                self.X_observed.append(X_vec)
                self.y_observed.append(score)
                print(f"    Initial {i+1}/{self.n_initial_points}: Score = {score:.4f}")
        
        if len(self.X_observed) == 0:
            print("    No valid initial points found!")
            return None, []
        
        # Convert to arrays
        X_observed = np.array(self.X_observed)
        y_observed = np.array(self.y_observed)
        
        # Bayesian optimization loop
        for iteration in range(self.n_calls - self.n_initial_points):
            try:
                # Fit Gaussian Process
                kernel = Matern(length_scale=1.0, nu=2.5)
                self.gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, random_state=42)
                self.gp.fit(X_observed, y_observed)
                
                # Find next point to evaluate using acquisition function
                best_acquisition = -np.inf
                best_candidate = None
                
                # Sample candidates and evaluate acquisition function
                for _ in range(100):  # Sample 100 candidates
                    candidate_params = self._sample_parameters(param_space)
                    candidate_vec = self._params_to_vector(candidate_params, param_space)
                    
                    acquisition_val = self._acquisition_function(candidate_vec, self.gp)
                    
                    if acquisition_val > best_acquisition:
                        best_acquisition = acquisition_val
                        best_candidate = candidate_vec
                
                if best_candidate is not None:
                    # Evaluate the best candidate
                    best_params = self._vector_to_params(best_candidate, param_space)
                    score = objective_function(best_params)
                    
                    if score is not None:
                        self.X_observed.append(best_candidate)
                        self.y_observed.append(score)
                        X_observed = np.array(self.X_observed)
                        y_observed = np.array(self.y_observed)
                        
                        print(f"    Iteration {iteration+1}: Score = {score:.4f}, Acquisition = {best_acquisition:.6f}")
                
            except Exception as e:
                print(f"    Iteration {iteration+1} failed: {str(e)}")
                # Fallback to random sampling
                params = self._sample_parameters(param_space)
                score = objective_function(params)
                if score is not None:
                    X_vec = self._params_to_vector(params, param_space)
                    self.X_observed.append(X_vec)
                    self.y_observed.append(score)
                    X_observed = np.array(self.X_observed)
                    y_observed = np.array(self.y_observed)
        
        # Find best result
        if len(self.y_observed) > 0:
            best_idx = np.argmax(self.y_observed)
            best_score = self.y_observed[best_idx]
            best_params = self._vector_to_params(self.X_observed[best_idx], param_space)
            
            return best_params, best_score
        else:
            return None, None

# Test Bayesian optimization
print("\n--- Testing Bayesian Optimization ---")

# Define objective function
def rf_objective(params):
    """Objective function for Random Forest optimization."""
    try:
        model = RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params.get('max_depth'),
            min_samples_split=params['min_samples_split'],
            random_state=42
        )
        
        # Cross-validation score
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
        return scores.mean()
        
    except Exception as e:
        return None

# Define parameter space for Bayesian optimization
bayesian_param_space = {
    'n_estimators': (10, 100),
    'max_depth': (3, 20),
    'min_samples_split': (2, 20)
}

# Run Bayesian optimization
bayesian_optimizer = SimpleBayesianOptimizer(n_initial_points=5, n_calls=15)
best_params, best_score = bayesian_optimizer.optimize(rf_objective, bayesian_param_space)

if best_params:
    print(f"\n🏆 Bayesian Optimization Results:")
    print(f"  Best Score: {best_score:.4f}")
    print(f"  Best Parameters: {best_params}")
    
    # Test final model
    final_model = RandomForestClassifier(**best_params, random_state=42)
    final_model.fit(X_train, y_train)
    test_score = final_model.score(X_test, y_test)
    print(f"  Test Score: {test_score:.4f}")
    
    # Save Bayesian optimized model
    bayesian_results = {
        'best_cv_score': best_score,
        'test_score': test_score,
        'optimization_method': 'bayesian',
        'n_iterations': len(bayesian_optimizer.y_observed),
        'convergence_history': bayesian_optimizer.y_observed
    }
    
    save_optimized_model(final_model, 'bayesian_random_forest',
                       'Random Forest optimized using Bayesian optimization', bayesian_results)
    
    # Save optimization history
    save_optimization_experiment('bayesian_optimization_history', {
        'parameter_space': bayesian_param_space,
        'optimization_history': [
            {'iteration': i, 'score': score} 
            for i, score in enumerate(bayesian_optimizer.y_observed)
        ],
        'best_params': best_params,
        'best_score': best_score
    }, 'Complete history of Bayesian optimization process')

else:
    print("❌ Bayesian optimization failed")

print("\n✨ Bayesian optimization complete!")

## 6. Population-Based Training {#population}

Implementing population-based training for parallel hyperparameter optimization.

In [ ]:
# Population-based training implementation
print("👥 Population-Based Training...")

class PopulationBasedTrainer:
    """Population-based training for hyperparameter optimization."""
    
    def __init__(self, population_size=6, generations=5, mutation_rate=0.3):
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.population = []
        self.fitness_history = []
    
    def _initialize_population(self, param_space):
        """Initialize random population."""
        population = []
        
        for _ in range(self.population_size):
            individual = {}
            for param_name, param_range in param_space.items():
                if isinstance(param_range, tuple):
                    # Continuous parameter
                    value = np.random.uniform(param_range[0], param_range[1])
                    if param_name in ['n_estimators', 'max_depth', 'min_samples_split']:
                        value = int(round(value))
                    individual[param_name] = value
                elif isinstance(param_range, list):
                    # Categorical parameter
                    individual[param_name] = np.random.choice(param_range)
            
            population.append(individual)
        
        return population
    
    def _evaluate_individual(self, individual, model_class, X_train, y_train):
        """Evaluate fitness of an individual."""
        try:
            model = model_class(**individual, random_state=42)
            scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
            return scores.mean()
        except Exception as e:
            return 0.0  # Poor fitness for invalid configurations
    
    def _mutate_individual(self, individual, param_space):
        """Mutate an individual by randomly changing some parameters."""
        mutated = individual.copy()
        
        for param_name, param_range in param_space.items():
            if np.random.random() < self.mutation_rate:
                if isinstance(param_range, tuple):
                    # Continuous parameter - add Gaussian noise
                    current_value = mutated[param_name]
                    noise_std = (param_range[1] - param_range[0]) * 0.1
                    new_value = current_value + np.random.normal(0, noise_std)
                    new_value = np.clip(new_value, param_range[0], param_range[1])
                    
                    if param_name in ['n_estimators', 'max_depth', 'min_samples_split']:
                        new_value = int(round(new_value))
                    
                    mutated[param_name] = new_value
                elif isinstance(param_range, list):
                    # Categorical parameter - random choice
                    mutated[param_name] = np.random.choice(param_range)
        
        return mutated
    
    def _crossover(self, parent1, parent2):
        """Create offspring through crossover."""
        child = {}
        for param_name in parent1.keys():
            # Random selection from either parent
            child[param_name] = parent1[param_name] if np.random.random() < 0.5 else parent2[param_name]
        return child
    
    def optimize(self, model_class, param_space, X_train, y_train):
        """Run population-based training."""
        print(f"  Initializing population of {self.population_size} individuals...")
        
        # Initialize population
        self.population = self._initialize_population(param_space)
        generation_fitness = []
        
        for generation in range(self.generations):
            print(f"\n  Generation {generation + 1}/{self.generations}")
            
            # Evaluate fitness for each individual
            fitness_scores = []
            for i, individual in enumerate(self.population):
                fitness = self._evaluate_individual(individual, model_class, X_train, y_train)
                fitness_scores.append(fitness)
                print(f"    Individual {i+1}: Fitness = {fitness:.4f}")
            
            generation_fitness.append(fitness_scores)
            self.fitness_history.extend(fitness_scores)
            
            # Selection: keep top 50% of population
            population_with_fitness = list(zip(self.population, fitness_scores))
            population_with_fitness.sort(key=lambda x: x[1], reverse=True)
            
            survivors = [individual for individual, _ in population_with_fitness[:self.population_size//2]]
            
            # Create new generation
            new_population = survivors.copy()
            
            # Fill remaining spots with offspring and mutations
            while len(new_population) < self.population_size:
                if len(survivors) >= 2:
                    # Crossover
                    parent1, parent2 = np.random.choice(survivors, 2, replace=False)
                    child = self._crossover(parent1, parent2)
                    
                    # Mutate child
                    child = self._mutate_individual(child, param_space)
                    new_population.append(child)
                else:
                    # Fallback: add random individual
                    new_individual = self._initialize_population(param_space)[0]
                    new_population.append(new_individual)
            
            self.population = new_population
            
            # Print generation statistics
            best_fitness = max(fitness_scores)
            avg_fitness = np.mean(fitness_scores)
            print(f"    Generation Stats: Best = {best_fitness:.4f}, Avg = {avg_fitness:.4f}")
        
        # Return best individual
        final_fitness_scores = []
        for individual in self.population:
            fitness = self._evaluate_individual(individual, model_class, X_train, y_train)
            final_fitness_scores.append(fitness)
        
        best_idx = np.argmax(final_fitness_scores)
        best_individual = self.population[best_idx]
        best_fitness = final_fitness_scores[best_idx]
        
        return best_individual, best_fitness, generation_fitness

# Test Population-Based Training
print("\n--- Testing Population-Based Training ---")

# Define parameter space for population-based training
pbt_param_space = {
    'n_estimators': (20, 100),
    'max_depth': (3, 15),
    'min_samples_split': (2, 10),
    'max_features': ['sqrt', 'log2']
}

# Run Population-Based Training
pbt_trainer = PopulationBasedTrainer(population_size=6, generations=4, mutation_rate=0.3)
best_individual, best_fitness, generation_fitness = pbt_trainer.optimize(
    RandomForestClassifier, pbt_param_space, X_train, y_train
)

print(f"\n🏆 Population-Based Training Results:")
print(f"  Best Fitness: {best_fitness:.4f}")
print(f"  Best Individual: {best_individual}")

# Test final model
final_model = RandomForestClassifier(**best_individual, random_state=42)
final_model.fit(X_train, y_train)
test_score = final_model.score(X_test, y_test)
print(f"  Test Score: {test_score:.4f}")

# Save PBT results
pbt_results = {
    'best_fitness': best_fitness,
    'test_score': test_score,
    'optimization_method': 'population_based_training',
    'population_size': pbt_trainer.population_size,
    'generations': pbt_trainer.generations,
    'fitness_history': pbt_trainer.fitness_history,
    'generation_fitness': generation_fitness
}

save_optimized_model(final_model, 'pbt_random_forest',
                   'Random Forest optimized using Population-Based Training', pbt_results)

print("\n✨ Population-based training complete!")

### Population-Based Training Visualization

In [ ]:
# Visualize population-based training evolution
print("📊 Visualizing Population-Based Training Evolution...")

if 'generation_fitness' in locals() and generation_fitness:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Fitness evolution over generations
    generations = range(1, len(generation_fitness) + 1)
    best_fitness_per_gen = [max(gen_fitness) for gen_fitness in generation_fitness]
    avg_fitness_per_gen = [np.mean(gen_fitness) for gen_fitness in generation_fitness]
    worst_fitness_per_gen = [min(gen_fitness) for gen_fitness in generation_fitness]
    
    axes[0, 0].plot(generations, best_fitness_per_gen, 'g-o', label='Best', linewidth=2)
    axes[0, 0].plot(generations, avg_fitness_per_gen, 'b-s', label='Average', linewidth=2)
    axes[0, 0].plot(generations, worst_fitness_per_gen, 'r-^', label='Worst', linewidth=2)
    axes[0, 0].set_xlabel('Generation')
    axes[0, 0].set_ylabel('Fitness (Accuracy)')
    axes[0, 0].set_title('Fitness Evolution Over Generations')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Population diversity over generations
    diversity_per_gen = []
    for gen_fitness in generation_fitness:
        diversity = np.std(gen_fitness) if len(gen_fitness) > 1 else 0
        diversity_per_gen.append(diversity)
    
    axes[0, 1].plot(generations, diversity_per_gen, 'purple', marker='o', linewidth=2)
    axes[0, 1].set_xlabel('Generation')
    axes[0, 1].set_ylabel('Fitness Diversity (Std Dev)')
    axes[0, 1].set_title('Population Diversity Over Generations')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Fitness distribution in final generation
    final_generation_fitness = generation_fitness[-1]
    axes[1, 0].hist(final_generation_fitness, bins=min(10, len(final_generation_fitness)), 
                   alpha=0.7, color='skyblue', edgecolor='black')
    axes[1, 0].axvline(np.mean(final_generation_fitness), color='red', linestyle='--', 
                      label=f'Mean: {np.mean(final_generation_fitness):.3f}')
    axes[1, 0].axvline(max(final_generation_fitness), color='green', linestyle='--', 
                      label=f'Best: {max(final_generation_fitness):.3f}')
    axes[1, 0].set_xlabel('Fitness (Accuracy)')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Final Generation Fitness Distribution')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Convergence analysis
    # Calculate improvement rate
    improvement_rates = []
    for i in range(1, len(best_fitness_per_gen)):
        improvement = best_fitness_per_gen[i] - best_fitness_per_gen[i-1]
        improvement_rates.append(improvement)
    
    if improvement_rates:
        axes[1, 1].bar(range(2, len(generations) + 1), improvement_rates, 
                      alpha=0.7, color='orange')
        axes[1, 1].set_xlabel('Generation')
        axes[1, 1].set_ylabel('Fitness Improvement')
        axes[1, 1].set_title('Fitness Improvement per Generation')
        axes[1, 1].grid(True, alpha=0.3)
        
        # Add zero line
        axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
    
    plt.tight_layout()
    
    # Save PBT evolution visualization
    save_figure(fig, 'population_based_training_evolution',
               'Evolution of population-based training showing fitness progression', 'pbt')
    plt.show()
    
    # PBT Summary
    print("\n📊 Population-Based Training Summary:")
    print("=" * 60)
    print(f"Initial Best Fitness: {best_fitness_per_gen[0]:.4f}")
    print(f"Final Best Fitness: {best_fitness_per_gen[-1]:.4f}")
    print(f"Total Improvement: {best_fitness_per_gen[-1] - best_fitness_per_gen[0]:.4f}")
    print(f"Average Improvement per Generation: {np.mean(improvement_rates):.4f}")
    print(f"Final Population Diversity: {diversity_per_gen[-1]:.4f}")
    print("=" * 60)

print("\n✨ Population-based training visualization complete!")

## 7. Cross-Validation Strategies {#cross-validation}

Exploring advanced cross-validation techniques for robust model evaluation.

In [ ]:
# Advanced cross-validation strategies
print("🔄 Advanced Cross-Validation Strategies...")

class AdvancedCrossValidator:
    """Advanced cross-validation techniques for robust model evaluation."""
    
    def __init__(self):
        self.cv_strategies = {
            'Stratified K-Fold': StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            'Group K-Fold': None,  # Will be set based on groups
            'Time Series Split': TimeSeriesSplit(n_splits=5),
            'Leave-One-Group-Out': None  # Will be set based on groups
        }
        self.results = {}
    
    def evaluate_cv_strategies(self, models, X, y, groups=None):
        """Evaluate different cross-validation strategies."""
        results = {}
        
        for model_name, model in models.items():
            print(f"\n  Evaluating {model_name}...")
            results[model_name] = {}
            
            for cv_name, cv_strategy in self.cv_strategies.items():
                try:
                    # Skip group-based CV if no groups provided
                    if cv_name in ['Group K-Fold', 'Leave-One-Group-Out'] and groups is None:
                        print(f"    {cv_name}: Skipped (no groups provided)")
                        continue
                    
                    # Set up group-based CV strategies
                    if cv_name == 'Group K-Fold' and groups is not None:
                        cv_strategy = GroupKFold(n_splits=min(5, len(np.unique(groups))))
                    elif cv_name == 'Leave-One-Group-Out' and groups is not None:
                        from sklearn.model_selection import LeaveOneGroupOut
                        cv_strategy = LeaveOneGroupOut()
                    
                    start_time = time.time()
                    
                    if groups is not None and cv_name in ['Group K-Fold', 'Leave-One-Group-Out']:
                        scores = cross_val_score(model, X, y, cv=cv_strategy, groups=groups, 
                                               scoring='accuracy', n_jobs=-1)
                    else:
                        scores = cross_val_score(model, X, y, cv=cv_strategy, 
                                               scoring='accuracy', n_jobs=-1)
                    
                    evaluation_time = time.time() - start_time
                    
                    results[model_name][cv_name] = {
                        'mean_score': scores.mean(),
                        'std_score': scores.std(),
                        'scores': scores.tolist(),
                        'evaluation_time': evaluation_time,
                        'n_splits': len(scores)
                    }
                    
                    print(f"    {cv_name}: {scores.mean():.4f} ± {scores.std():.4f}")
                    
                except Exception as e:
                    print(f"    {cv_name}: Failed ({str(e)})")
                    results[model_name][cv_name] = {'error': str(e)}
        
        return results
    
    def learning_curve_analysis(self, model, X, y, cv_strategy=None):
        """Analyze learning curves with different training set sizes."""
        if cv_strategy is None:
            cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        train_sizes = np.linspace(0.1, 1.0, 10)
        
        train_sizes_abs, train_scores, val_scores = learning_curve(
            model, X, y, train_sizes=train_sizes, cv=cv_strategy,
            scoring='accuracy', n_jobs=-1, random_state=42
        )
        
        return {
            'train_sizes': train_sizes_abs,
            'train_scores_mean': train_scores.mean(axis=1),
            'train_scores_std': train_scores.std(axis=1),
            'val_scores_mean': val_scores.mean(axis=1),
            'val_scores_std': val_scores.std(axis=1)
        }
    
    def validation_curve_analysis(self, model, X, y, param_name, param_range, cv_strategy=None):
        """Analyze validation curves for hyperparameter sensitivity."""
        if cv_strategy is None:
            cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        train_scores, val_scores = validation_curve(
            model, X, y, param_name=param_name, param_range=param_range,
            cv=cv_strategy, scoring='accuracy', n_jobs=-1
        )
        
        return {
            'param_range': param_range,
            'train_scores_mean': train_scores.mean(axis=1),
            'train_scores_std': train_scores.std(axis=1),
            'val_scores_mean': val_scores.mean(axis=1),
            'val_scores_std': val_scores.std(axis=1)
        }

# Test advanced cross-validation
print("\n--- Testing Advanced Cross-Validation Strategies ---")

# Create synthetic groups for group-based CV
np.random.seed(42)
groups = np.random.randint(0, 10, size=len(X_train))

# Models to evaluate
cv_test_models = {
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=50, random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

# Initialize advanced cross-validator
cv_evaluator = AdvancedCrossValidator()

# Evaluate different CV strategies
cv_results = cv_evaluator.evaluate_cv_strategies(cv_test_models, X_train, y_train, groups)

# Learning curve analysis
print(f"\n  Analyzing learning curves...")
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)
learning_curves = cv_evaluator.learning_curve_analysis(rf_model, X_train, y_train)

# Validation curve analysis
print(f"  Analyzing validation curves...")
n_estimators_range = [10, 25, 50, 75, 100, 150, 200]
validation_curves = cv_evaluator.validation_curve_analysis(
    RandomForestClassifier(random_state=42), X_train, y_train,
    'n_estimators', n_estimators_range
)

print("\n✨ Advanced cross-validation analysis complete!")

# Save cross-validation results
cv_summary = {}
for model_name, cv_strategies in cv_results.items():
    cv_summary[model_name] = {}
    for cv_name, results in cv_strategies.items():
        if 'error' not in results:
            cv_summary[model_name][cv_name] = {
                'mean_score': results['mean_score'],
                'std_score': results['std_score'],
                'evaluation_time': results['evaluation_time'],
                'n_splits': results['n_splits']
            }

save_optimization_experiment('cross_validation_strategies', {
    'cv_strategy_comparison': cv_summary,
    'learning_curve_analysis': learning_curves,
    'validation_curve_analysis': validation_curves
}, 'Comprehensive analysis of cross-validation strategies and learning curves')

### Cross-Validation Visualization

In [ ]:
# Visualize cross-validation results
print("📊 Visualizing Cross-Validation Results...")

if cv_results:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. CV Strategy Comparison
    cv_comparison_data = []
    for model_name, cv_strategies in cv_results.items():
        for cv_name, results in cv_strategies.items():
            if 'error' not in results:
                cv_comparison_data.append({
                    'Model': model_name,
                    'CV_Strategy': cv_name,
                    'Mean_Score': results['mean_score'],
                    'Std_Score': results['std_score']
                })
    
    if cv_comparison_data:
        cv_df = pd.DataFrame(cv_comparison_data)
        
        # Group by CV strategy for better visualization
        pivot_means = cv_df.pivot(index='CV_Strategy', columns='Model', values='Mean_Score')
        pivot_stds = cv_df.pivot(index='CV_Strategy', columns='Model', values='Std_Score')
        
        # Bar plot with error bars
        x_pos = np.arange(len(pivot_means.index))
        width = 0.25
        
        for i, model in enumerate(pivot_means.columns):
            means = pivot_means[model].values
            stds = pivot_stds[model].values
            axes[0, 0].bar(x_pos + i * width, means, width, yerr=stds, 
                          label=model, alpha=0.7, capsize=5)
        
        axes[0, 0].set_xlabel('Cross-Validation Strategy')
        axes[0, 0].set_ylabel('Accuracy')
        axes[0, 0].set_title('Cross-Validation Strategy Comparison')
        axes[0, 0].set_xticks(x_pos + width)
        axes[0, 0].set_xticklabels(pivot_means.index, rotation=45, ha='right')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Learning Curves
    if 'learning_curves' in locals():
        lc = learning_curves
        axes[0, 1].plot(lc['train_sizes'], lc['train_scores_mean'], 'o-', color='blue', 
                       label='Training Score')
        axes[0, 1].fill_between(lc['train_sizes'], 
                               lc['train_scores_mean'] - lc['train_scores_std'],
                               lc['train_scores_mean'] + lc['train_scores_std'], 
                               alpha=0.3, color='blue')
        
        axes[0, 1].plot(lc['train_sizes'], lc['val_scores_mean'], 'o-', color='red', 
                       label='Validation Score')
        axes[0, 1].fill_between(lc['train_sizes'], 
                               lc['val_scores_mean'] - lc['val_scores_std'],
                               lc['val_scores_mean'] + lc['val_scores_std'], 
                               alpha=0.3, color='red')
        
        axes[0, 1].set_xlabel('Training Set Size')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].set_title('Learning Curves')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Validation Curves
    if 'validation_curves' in locals():
        vc = validation_curves
        axes[1, 0].plot(vc['param_range'], vc['train_scores_mean'], 'o-', color='blue', 
                       label='Training Score')
        axes[1, 0].fill_between(vc['param_range'], 
                               vc['train_scores_mean'] - vc['train_scores_std'],
                               vc['train_scores_mean'] + vc['train_scores_std'], 
                               alpha=0.3, color='blue')
        
        axes[1, 0].plot(vc['param_range'], vc['val_scores_mean'], 'o-', color='red', 
                       label='Validation Score')
        axes[1, 0].fill_between(vc['param_range'], 
                               vc['val_scores_mean'] - vc['val_scores_std'],
                               vc['val_scores_mean'] + vc['val_scores_std'], 
                               alpha=0.3, color='red')
        
        axes[1, 0].set_xlabel('n_estimators')
        axes[1, 0].set_ylabel('Accuracy')
        axes[1, 0].set_title('Validation Curves (n_estimators)')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. CV Strategy Reliability (variance comparison)
    if cv_comparison_data:
        strategy_variance = cv_df.groupby('CV_Strategy')['Std_Score'].mean().sort_values()
        
        bars = axes[1, 1].bar(range(len(strategy_variance)), strategy_variance.values, 
                             alpha=0.7, color='lightgreen')
        axes[1, 1].set_xlabel('Cross-Validation Strategy')
        axes[1, 1].set_ylabel('Average Standard Deviation')
        axes[1, 1].set_title('CV Strategy Reliability (Lower = More Stable)')
        axes[1, 1].set_xticks(range(len(strategy_variance)))
        axes[1, 1].set_xticklabels(strategy_variance.index, rotation=45, ha='right')
        axes[1, 1].grid(True, alpha=0.3)
        
        # Add value labels
        for bar, value in zip(bars, strategy_variance.values):
            axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                           f'{value:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    
    # Save cross-validation visualization
    save_figure(fig, 'cross_validation_analysis',
               'Comprehensive analysis of cross-validation strategies and learning curves', 'cv')
    plt.show()
    
    # CV Summary
    print("\n📊 Cross-Validation Analysis Summary:")
    print("=" * 80)
    
    if cv_comparison_data:
        print("Best performing CV strategies by model:")
        for model in cv_df['Model'].unique():
            model_data = cv_df[cv_df['Model'] == model]
            best_cv = model_data.loc[model_data['Mean_Score'].idxmax()]
            print(f"  {model}: {best_cv['CV_Strategy']} (Score: {best_cv['Mean_Score']:.4f})")
        
        print(f"\nMost reliable CV strategy: {strategy_variance.index[0]} (Std: {strategy_variance.iloc[0]:.4f})")
        print(f"Least reliable CV strategy: {strategy_variance.index[-1]} (Std: {strategy_variance.iloc[-1]:.4f})")
    
    print("=" * 80)

print("\n✨ Cross-validation visualization complete!")

## 8. Model Selection Pipelines {#pipelines}

Building comprehensive model selection pipelines that combine multiple techniques.

In [ ]:
# Comprehensive model selection pipeline
print("🔧 Building Model Selection Pipelines...")

class ComprehensiveModelSelectionPipeline:
    """Complete pipeline for automated model selection and optimization."""
    
    def __init__(self, optimization_budget=100, cv_folds=5, test_size=0.2):
        self.optimization_budget = optimization_budget
        self.cv_folds = cv_folds
        self.test_size = test_size
        self.results = {}
        self.best_model = None
        self.pipeline_history = []
    
    def run_pipeline(self, X, y, task_type='classification'):
        """Run the complete model selection pipeline."""
        print("🚀 Starting Comprehensive Model Selection Pipeline...")
        
        # Stage 1: Data preparation and splitting
        print("\n📊 Stage 1: Data Preparation")
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=42, 
            stratify=y if task_type == 'classification' else None
        )
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=0.25, random_state=42,
            stratify=y_temp if task_type == 'classification' else None
        )
        
        print(f"  Training set: {X_train.shape}")
        print(f"  Validation set: {X_val.shape}")
        print(f"  Test set: {X_test.shape}")
        
        # Stage 2: Quick model screening
        print("\n🔍 Stage 2: Quick Model Screening")
        screening_results = self._quick_model_screening(X_train, y_train)
        
        # Stage 3: Hyperparameter optimization for top models
        print("\n⚙️ Stage 3: Hyperparameter Optimization")
        optimization_results = self._optimize_top_models(
            screening_results, X_train, y_train, X_val, y_val
        )
        
        # Stage 4: Advanced validation
        print("\n🔄 Stage 4: Advanced Cross-Validation")
        validation_results = self._advanced_validation(
            optimization_results, X_train, y_train
        )
        
        # Stage 5: Final model selection and testing
        print("\n🏆 Stage 5: Final Model Selection")
        final_results = self._final_model_selection(
            validation_results, X_train, y_train, X_test, y_test
        )
        
        # Generate comprehensive report
        report = self._generate_pipeline_report(final_results)
        
        return final_results, report
    
    def _quick_model_screening(self, X_train, y_train):
        """Quickly screen multiple models with default parameters."""
        candidate_models = {
            'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
            'RandomForest': RandomForestClassifier(random_state=42),
            'GradientBoosting': GradientBoostingClassifier(random_state=42),
            'SVM': SVC(random_state=42),
            'MLPClassifier': MLPClassifier(random_state=42, max_iter=500)
        }
        
        screening_results = {}
        
        for name, model in candidate_models.items():
            try:
                start_time = time.time()
                scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
                screening_time = time.time() - start_time
                
                screening_results[name] = {
                    'mean_score': scores.mean(),
                    'std_score': scores.std(),
                    'screening_time': screening_time,
                    'model': model
                }
                
                print(f"  {name}: {scores.mean():.4f} ± {scores.std():.4f} ({screening_time:.2f}s)")
                
            except Exception as e:
                print(f"  {name}: Failed ({str(e)})")
        
        # Select top 3 models for optimization
        valid_results = {k: v for k, v in screening_results.items() if 'model' in v}
        top_models = sorted(valid_results.items(), key=lambda x: x[1]['mean_score'], reverse=True)[:3]
        
        print(f"\n  Selected top {len(top_models)} models for optimization:")
        for name, result in top_models:
            print(f"    {name}: {result['mean_score']:.4f}")
        
        return dict(top_models)
    
    def _optimize_top_models(self, top_models, X_train, y_train, X_val, y_val):
        """Optimize hyperparameters for top performing models."""
        optimization_results = {}
        
        # Define parameter spaces
        param_spaces = {
            'RandomForest': {
                'n_estimators': randint(50, 200),
                'max_depth': randint(5, 20),
                'min_samples_split': randint(2, 10)
            },
            'GradientBoosting': {
                'n_estimators': randint(50, 150),
                'learning_rate': uniform(0.05, 0.25),
                'max_depth': randint(3, 8)
            },
            'LogisticRegression': {
                'C': uniform(0.1, 10),
                'penalty': ['l1', 'l2'],
                'solver': ['liblinear', 'saga']
            },
            'SVM': {
                'C': uniform(0.1, 10),
                'gamma': ['scale'] + list(uniform(0.001, 1).rvs(3)),
                'kernel': ['rbf', 'poly']
            },
            'MLPClassifier': {
                'hidden_layer_sizes': [(50,), (100,), (50, 25), (100, 50)],
                'alpha': uniform(0.0001, 0.01),
                'learning_rate_init': uniform(0.001, 0.01)
            }
        }
        
        for model_name, model_info in top_models.items():
            if model_name in param_spaces:
                print(f"  Optimizing {model_name}...")
                
                try:
                    # Random search optimization
                    search = RandomizedSearchCV(
                        estimator=model_info['model'],
                        param_distributions=param_spaces[model_name],
                        n_iter=20,  # Reduced for demonstration
                        cv=3,
                        scoring='accuracy',
                        random_state=42,
                        n_jobs=-1
                    )
                    
                    start_time = time.time()
                    search.fit(X_train, y_train)
                    optimization_time = time.time() - start_time
                    
                    # Validate on holdout set
                    val_score = search.best_estimator_.score(X_val, y_val)
                    
                    optimization_results[model_name] = {
                        'best_estimator': search.best_estimator_,
                        'best_params': search.best_params_,
                        'best_cv_score': search.best_score_,
                        'val_score': val_score,
                        'optimization_time': optimization_time
                    }
                    
                    print(f"    CV Score: {search.best_score_:.4f}")
                    print(f"    Val Score: {val_score:.4f}")
                    print(f"    Time: {optimization_time:.2f}s")
                    
                except Exception as e:
                    print(f"    Failed: {str(e)}")
            else:
                # Keep original model if no parameter space defined
                optimization_results[model_name] = {
                    'best_estimator': model_info['model'],
                    'best_params': {},
                    'best_cv_score': model_info['mean_score'],
                    'val_score': model_info['model'].fit(X_train, y_train).score(X_val, y_val),
                    'optimization_time': 0
                }
        
        return optimization_results
    
    def _advanced_validation(self, optimization_results, X_train, y_train):
        """Perform advanced cross-validation on optimized models."""
        validation_results = {}
        
        # Different CV strategies
        cv_strategies = {
            'StratifiedKFold': StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            'RepeatedStratifiedKFold': None  # Simplified for demo
        }
        
        for model_name, model_info in optimization_results.items():
            print(f"  Validating {model_name}...")
            
            validation_results[model_name] = model_info.copy()
            validation_results[model_name]['cv_validation'] = {}
            
            model = model_info['best_estimator']
            
            for cv_name, cv_strategy in cv_strategies.items():
                if cv_strategy is not None:
                    try:
                        scores = cross_val_score(model, X_train, y_train, cv=cv_strategy, 
                                               scoring='accuracy', n_jobs=-1)
                        
                        validation_results[model_name]['cv_validation'][cv_name] = {
                            'mean_score': scores.mean(),
                            'std_score': scores.std(),
                            'scores': scores.tolist()
                        }
                        
                        print(f"    {cv_name}: {scores.mean():.4f} ± {scores.std():.4f}")
                        
                    except Exception as e:
                        print(f"    {cv_name}: Failed ({str(e)})")
        
        return validation_results
    
    def _final_model_selection(self, validation_results, X_train, y_train, X_test, y_test):
        """Select final model and evaluate on test set."""
        # Select best model based on validation score
        best_model_name = None
        best_score = 0
        
        for model_name, results in validation_results.items():
            cv_results = results.get('cv_validation', {})
            if 'StratifiedKFold' in cv_results:
                score = cv_results['StratifiedKFold']['mean_score']
                if score > best_score:
                    best_score = score
                    best_model_name = model_name
        
        if best_model_name:
            print(f"  Selected best model: {best_model_name}")
            
            # Train final model on full training set
            best_model = validation_results[best_model_name]['best_estimator']
            best_model.fit(X_train, y_train)
            
            # Final test evaluation
            test_score = best_model.score(X_test, y_test)
            y_pred = best_model.predict(X_test)
            
            print(f"  Final test score: {test_score:.4f}")
            
            final_results = {
                'best_model_name': best_model_name,
                'best_model': best_model,
                'best_params': validation_results[best_model_name]['best_params'],
                'cv_score': best_score,
                'test_score': test_score,
                'y_pred': y_pred,
                'y_test': y_test,
                'all_validation_results': validation_results
            }
            
            self.best_model = best_model
            
            return final_results
        else:
            print("  No valid model found!")
            return None
    
    def _generate_pipeline_report(self, final_results):
        """Generate comprehensive pipeline report."""
        if not final_results:
            return "Pipeline failed - no valid results."
        
        report = "\n" + "="*80 + "\n"
        report += "COMPREHENSIVE MODEL SELECTION PIPELINE REPORT\n"
        report += "="*80 + "\n\n"
        
        report += f"Best Model: {final_results['best_model_name']}\n"
        report += f"Best Parameters: {final_results['best_params']}\n"
        report += f"Cross-Validation Score: {final_results['cv_score']:.4f}\n"
        report += f"Final Test Score: {final_results['test_score']:.4f}\n\n"
        
        report += "Model Comparison Summary:\n"
        report += "-" * 40 + "\n"
        
        for model_name, results in final_results['all_validation_results'].items():
            cv_results = results.get('cv_validation', {})
            if 'StratifiedKFold' in cv_results:
                score = cv_results['StratifiedKFold']['mean_score']
                std = cv_results['StratifiedKFold']['std_score']
                report += f"{model_name}: {score:.4f} ± {std:.4f}\n"
        
        report += "\n" + "="*80 + "\n"
        
        return report

# Run the comprehensive pipeline
print("\n--- Running Comprehensive Model Selection Pipeline ---")

# Use the complex dataset
X_pipeline, y_pipeline = datasets['Complex'][0], datasets['Complex'][1]
X_combined = np.vstack([X_pipeline, datasets['Complex'][2]])
y_combined = np.hstack([y_pipeline, datasets['Complex'][3]])

# Initialize and run pipeline
pipeline = ComprehensiveModelSelectionPipeline(
    optimization_budget=100,
    cv_folds=5,
    test_size=0.2
)

try:
    final_results, pipeline_report = pipeline.run_pipeline(X_combined, y_combined)
    
    if final_results:
        print(pipeline_report)
        
        # Save pipeline results
        pipeline_summary = {
            'best_model_name': final_results['best_model_name'],
            'best_params': final_results['best_params'],
            'cv_score': final_results['cv_score'],
            'test_score': final_results['test_score'],
            'pipeline_stages': ['screening', 'optimization', 'validation', 'selection']
        }
        
        save_optimization_experiment('comprehensive_pipeline_results', pipeline_summary,
                                   'Results from comprehensive model selection pipeline')
        
        # Save the final best model
        save_optimized_model(final_results['best_model'], 'pipeline_best_model',
                           'Best model selected through comprehensive pipeline', pipeline_summary)
        
        # Save the pipeline itself
        save_pipeline(pipeline, 'comprehensive_model_selection',
                     'Complete model selection pipeline with all stages', pipeline_summary)
        
        # Save the report
        save_model_selection_report(pipeline_report, 'comprehensive_pipeline_report', 'txt')
        
    else:
        print("❌ Pipeline failed to produce results")
        
except Exception as e:
    print(f"❌ Pipeline failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("\n✨ Comprehensive model selection pipeline complete!")

## Conclusion

This comprehensive notebook has demonstrated advanced model selection and hyperparameter tuning techniques including:

- **Automated Model Selection**: Intelligent algorithm selection based on dataset characteristics
- **Advanced Hyperparameter Optimization**: Grid search, random search, and custom optimization strategies
- **Multi-Objective Optimization**: Balancing accuracy, speed, and model complexity
- **Bayesian Optimization**: Efficient hyperparameter search using Gaussian processes
- **Population-Based Training**: Evolutionary approaches to hyperparameter optimization
- **Advanced Cross-Validation**: Robust evaluation strategies for different data scenarios
- **Comprehensive Pipelines**: End-to-end model selection workflows

### Key Features

1. **Intelligent Automation**: Automated model selection based on dataset characteristics
2. **Multi-Strategy Optimization**: Comparison of different optimization approaches
3. **Robust Validation**: Advanced cross-validation techniques for reliable evaluation
4. **Comprehensive Reporting**: Detailed analysis and comparison of results
5. **Production Ready**: Complete pipelines with monitoring and result tracking

### Next Steps

1. **Integrate with AutoML frameworks** for even more sophisticated automation
2. **Add neural architecture search** for deep learning models
3. **Implement distributed optimization** for large-scale hyperparameter search
4. **Add cost-aware optimization** considering computational budgets
5. **Extend to multi-modal datasets** and specialized domains

The modular design allows for easy extension and customization for specific use cases, while the comprehensive result tracking ensures reproducibility and analysis of optimization strategies.